# Digit Classification with Neural Networks (MNIST)

Welcome to this machine learning project!  
In this notebook, we’ll build a **2-layer neural network from scratch using NumPy** to classify **handwritten digits** from the popular **MNIST dataset**.


## What You’ll Learn

- How to load and preprocess image data
- How neural networks work under the hood (no black boxes!)
- How to implement forward propagation with ReLU and Softmax
- How to compute categorical cross-entropy loss
- How to implement backpropagation and train the model
- How to evaluate a model and visualize predictions


## What Is MNIST?

- 70,000 grayscale images of handwritten digits (0–9)
- Each image is **28 × 28 pixels**
- Goal: Predict the correct digit label for each image


## Why This Project Matters

Understanding how neural networks work **under the hood** is essential for any machine learning practitioner.

Instead of relying on high-level libraries like TensorFlow or scikit-learn, this project builds everything from scratch using NumPy — layer by layer, function by function.

By doing so, we develop a clear intuition for:
- How neural networks process data through layers
- What activation functions really do
- How predictions turn into probabilities
- How loss functions and gradients guide learning

This hands-on approach helps bridge the gap between **theory and application**, and lays a solid foundation for tackling more complex models and real-world deep learning problems.


Let’s dive in!
